In [1]:
import jittor as jt
from PIL import Image
import jclip as clip
import os
from tqdm import tqdm
import argparse
from sklearn.linear_model import LogisticRegression
import numpy as np
import random
# from colorama import Fore, Back, Style, init
# init()

jt.flags.use_cuda = 1
print("包导入成功")
parser = argparse.ArgumentParser()
parser.add_argument('--split', type=str, default='A')

# args = parser.parse_args()
args, unknown = parser.parse_known_args()
model, preprocess = clip.load("ViT-B-32.pkl")
imgs_dir = '../data'

[i 0616 17:39:02.531531 40 log.cc:351] Load log_sync: 1
[i 0616 17:39:02.588374 40 compiler.py:956] Jittor(1.3.8.5) src: /root/miniconda3/envs/pyt/lib/python3.10/site-packages/jittor
[i 0616 17:39:02.590777 40 compiler.py:957] g++ at /usr/bin/g++(9.4.0)
[i 0616 17:39:02.591442 40 compiler.py:958] cache_path: /root/.cache/jittor/jt1.3.8/g++9.4.0/py3.10.11/Linux-5.4.0-16xb5/IntelRXeonRPlax40/default
[i 0616 17:39:02.594396 40 __init__.py:411] Found /usr/local/cuda/bin/nvcc(11.7.99) at /usr/local/cuda/bin/nvcc.
[i 0616 17:39:02.597084 40 __init__.py:411] Found addr2line(2.34) at /usr/bin/addr2line.


[i 0616 17:39:02.685681 40 compiler.py:1011] cuda key:cu11.7.99_sm_70
[i 0616 17:39:03.025154 40 __init__.py:227] Total mem: 38.42GB, using 12 procs for compiling.
[i 0616 17:39:03.166347 40 jit_compiler.cc:28] Load cc_path: /usr/bin/g++
[i 0616 17:39:03.233612 40 init.cc:62] Found cuda archs: [70,]
[i 0616 17:39:06.086123 40 cuda_flags.cc:49] CUDA enabled.


包导入成功


In [5]:

# encode这块后面可以强化下
def encode_pre_word(c)->str:
    """更具输入的名字返回一句话"""
    seq = 'a photo of ' + c
    return seq

def ret_class_name_dic()->dict:
    """返回数字映射到动物名字的字典"""
    classes = open('../data/classname.txt').read().splitlines()#这是一个包含所有类的列表
    class_name_dic={}#这是数字映射到动物名字的字典
    for i in classes:
        name,idx = i.split(' ')
        if idx not in class_name_dic:
            class_name_dic[idx]=name
    return class_name_dic

class_name_dic=ret_class_name_dic()

In [11]:
#训练的数据处理
def train_data()->list:
    """返回元素为玩意的列表['TrainSet/Animal/Bee/57.jpg', '1'] 每类四个"""
    train_labels = open('../data/train.txt').read().splitlines()
    train_data_dic={}#每类的图片的字典
    for i in train_labels:
        path,class_name=i.split(' ')
        if class_name in train_data_dic:
            train_data_dic[class_name].append([path,class_name])
        else:
            train_data_dic[class_name]=[]
    # 训练集的要训练的每类四张的列表
    ret_list=[]#用于返回每类四张的列表
    for i in train_data_dic:
        ret_list+=random.sample(train_data_dic[i],4)
    
    return ret_list

train_data=train_data()



In [12]:
len(train_data)

1496

In [13]:
# 把句子和图片进行encoding
train_img_features = []
train_word_features=[]
count=0
with jt.no_grad():
    class_name_dic=ret_class_name_dic()
    for info in tqdm(train_data):

        img,indx=info
        img = os.path.join(imgs_dir, img)
        image = Image.open(img)
        image = preprocess(image).unsqueeze(0)
        image_features = model.encode_image(image)
        # print("能成功运行?","image_features的shape是",image_features.shape)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        train_img_features.append(image_features)

        a_seq=encode_pre_word(class_name_dic[indx])#转为句子
        token=clip.tokenize(a_seq)
        
        text_features=model.encode_text(token)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        print("text_features的shape为",text_features.shape)
        train_word_features.append(text_features)
        count+=1
        if count==5 :

            break

train_features = jt.cat(train_img_features).numpy()#(1496, 512)
train_labels = jt.cat(train_word_features).numpy()#(1496,)

  0%|          | 4/1496 [00:00<00:43, 33.92it/s]

text_features的shape为 [1,512,]
text_features的shape为 [1,512,]
text_features的shape为 [1,512,]
text_features的shape为 [1,512,]
text_features的shape为 [1,512,]


In [10]:
print(train_features.shape)
print(train_labels.shape)

(1496, 512)


(1496, 512)

In [ ]:
#测试数据处理
def test_data()->list:
    """返回元素为玩意的列表['TrainSet/Animal/Bee/57.jpg 1] 共计3000个用于测试"""
    train_labels = open('../data/train.txt').read().splitlines() 
    return random.sample(train_labels,3000)